In [1]:
import sys
sys.path.append("..")
from shift_detector import *
from shift_reductor import *
from shift_experiments import *
from shift_tester import *
import pandas as pd
import os
from shift_plot_utils import *
from shift_utils import *

2022-03-03 08:51:13.595905: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/R/4.0.4/lib/R/lib::/lib:/usr/local/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/java-11-openjdk-amd64/lib/server
2022-03-03 08:51:13.595970: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-03-03 08:51:16.467475: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-03 08:51:16.467541: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cpu1.gemini-hpc.ca): /proc/driver/nvidia/version does not exist
2022-03-03 08:51:16.468912: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is 

###  Load data ###

In [2]:
print("Loading data...")
(X_s_tr, y_s_tr), (X_s_val, y_s_val), (X_t, y_t), orig_dims = import_dataset('los')

Loading data...


### Define Shift Detector and Parameters ###

In [3]:
sign_level = 0.05
dr_technique = "PCA"
md_test = "MMD"
red_model = None
datset = "test experiment"
samples = [10, 20, 50, 100, 200, 500, 1000,2000]
sample = samples[5]
orig_dims = X_s_tr.shape[1:]
print("Building shift detector...")
sd = ShiftDetector(dr_technique, md_test, sign_level, red_model, sample, datset)

Building shift detector...


#### Drift from 2019 to 2019 ####

In [49]:
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_tr, y_s_tr, X_s_val[:sample,:], y_s_val[:sample], orig_dims)
print(p_val,dist)

No GPU detected, fall back on CPU.
0.8799999952316284 -0.0017459988594055176


#### Drift from 2019 to 2020 ####

In [50]:
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_t[:sample,:], y_t[:sample], orig_dims)
print(p_val,dist)

No GPU detected, fall back on CPU.
0.019999999552965164 0.0064514875411987305


#### Knockout Shift ####

In [11]:
shift = "ko_shift_1.0"
X_ko, y_ko = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_ko[:sample,:], y_ko[:sample], orig_dims)
print(p_val,dist)

No GPU detected, fall back on CPU.
0.009999999776482582 0.007656693458557129


#### Gaussian Noise Shift ####

In [39]:
shift = "small_gn_shift_0.1"
X_sgn, y_sgn = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_sgn[:sample,:], y_sgn[:sample], orig_dims)
print(p_val,dist)

shift = "large_gn_shift_1.0"
X_lgn, y_lgn = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_lgn[:sample,:], y_lgn[:sample], orig_dims)
print(p_val,dist)

No GPU detected, fall back on CPU.
1.0 -0.002201557159423828
No GPU detected, fall back on CPU.
1.0 -3.039836883544922e-06


#### Multiway Feature Association Shift ####

In [40]:
shift = "mfa_shift_0.5"
X_mfa, y_mfa = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_mfa[:sample,:], y_mfa[:sample], orig_dims)
print(p_val,dist)

No GPU detected, fall back on CPU.
1.0 -3.039836883544922e-06


#### Changepoint Shift ####

In [41]:
shift= "cp_shift_0.75"
X_cp, y_cp = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_cp[:sample,:], y_cp[:sample], orig_dims)
print(p_val,dist)

/mnt/nfs/home/subasriv/.conda/envs/vector_delirium/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [16 20] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/mnt/nfs/home/subasriv/.conda/envs/vector_delirium/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


No GPU detected, fall back on CPU.
1.0 -3.039836883544922e-06
